In [ ]:
!pip install ckanapi
from ckanapi import *
import csv
import requests

api = RemoteCKAN('https://dadosabertos.capes.gov.br')
datasets = api.action.package_list()


In [ ]:
#Função para filtrar a lista de datasets, colocando os que se adequam ao filtro
def filterCatch(data, filter):
  return [str for str in data if
          any(sub in str for sub in filter)]

#Função para filtrar a lista de datasets, retirando os que se adequam ao filtro
def filterMiss(data, filter):
  return [str for str in data if
          not any(sub in str for sub in filter)]

In [ ]:
filters=["2021-a-2024"]
database=filterCatch(datasets,filters)
print(*database,sep="\n")

In [ ]:
filters=["artistica"]
database=filterMiss(database,filters)
print(*database,sep="\n")

In [ ]:
import logging
import time

import os

def download_dataset(database, caminho, max_attempts=3):
    for dataset in database:
        try:
            archive = api.action.package_show(id=dataset)
            link_archive = archive['resources'][0]['url']
            name_archive = caminho + dataset + '.csv'

            # Verificar se o arquivo já existe

            for attempt in range(max_attempts):
                try:
                    print(f"Tentativa {attempt + 1}/{max_attempts}")
                    !wget -O {name_archive} --retry-connrefused {link_archive}
                    break  # Download bem-sucedido

                except ConnectionError as e:
                    logging.error(f"Erro de conexão ao baixar dataset {dataset} (tentativa {attempt + 1}/{max_attempts}): {e}")
                    time.sleep(60)  # Aguardar 1 minuto antes de tentar novamente

                    # Tentar continuar com -c em caso de erro de conexão
                    !wget -c {name_archive} {link_archive}

                except Exception as e:
                    logging.error(f"Erro inesperado ao baixar dataset {dataset}: {e}")

        except Exception as e:
            logging.error(f"Erro ao obter informações do dataset {dataset}: {e}")

In [ ]:
download_dataset(database,'csv/')